In [1]:
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
from Curves import Curves
from ImportData import importSWEiopa
from BondClasses import ZeroCouponBond,ZeroCouponBondPriced

In [3]:
paramfile = pd.read_csv("Parameters.csv")
paramfile.index = paramfile["Parameter"]
del paramfile["Parameter"]

# IMPORT EIOPA CURVE

In [4]:
selected_param_file = paramfile.loc["EIOPA_param_file"][0]
selected_curves_file = paramfile.loc["EIOPA_curves_file"][0]
country = paramfile.loc["country"]["value"]
compounding = int(paramfile.loc["compounding"]["value"])

# Model parameters
MD = dt.datetime.strptime(paramfile.loc["Modelling_Date"]["value"],"%d/%m/%Y")
MD = dt.date(MD.year,MD.month,MD.day)

[maturities_country, curve_country, extra_param, Qb]= importSWEiopa(selected_param_file, selected_curves_file, country)

In [5]:
# Maturity of observations:
M_Obs = np.transpose(np.array(maturities_country.values))

# Ultimate froward rate ufr represents the rate to which the rate curve will converge as time increases:
ufr = extra_param.iloc[3]/100

# Convergence speed parameter alpha controls the speed at which the curve converges towards the ufr from the last liquid point:
alpha = extra_param.iloc[4]

# Qb calibration vector published by EIOPA for the curve calibration:
Qb = np.transpose(np.array(Qb.values))

# PRICING OF A COUPON BOND

In [6]:
bondfile = pd.read_csv("ZCB_Input.csv")
bondfile.index = bondfile["Asset ID"]
del bondfile["Asset ID"]
bondfile

,Asset_Type,Issue_Date,Maturity_Date,Notional_amount,Coupon_Rate,Frequency,Recovery_Rate,Default_Probability,Sector_Spread
Asset ID,,,,,,,,,
1,Corporate_Bond,3/12/2021,12/12/2026,100,0.03,1,0.4,0.03,0.008
2,Corporate_Bond,3/12/2021,12/12/2028,100,0.05,1,0.4,0.03,0.008
3,Corporate_Bond,3/12/2019,3/12/2020,100,0.04,1,0.4,0.03,0.008


In [7]:
def converttime(date_time):
    format = "%d/%m/%Y"
    datetime_str = dt.datetime.strptime(date_time,format).date()
    return datetime_str

In [8]:
# To Fix
issuedate = np.array([])
maturitydate = np.array([])
notional = np.array([])
couponrate = np.array([])
frequency = np.array([])
recovrate = np.array([])
defprob = np.array([])
sspread = np.array([])
zspread = np.array([])

for iCount in range(0,bondfile.index.size):
    issuedate = np.append(issuedate,converttime(bondfile["Issue_Date"].iloc[iCount]))
    maturitydate = np.append(maturitydate,converttime(bondfile["Maturity_Date"].iloc[iCount]))
    notional = np.append(notional,bondfile["Notional_amount"].iloc[iCount])
    couponrate = np.append(couponrate,bondfile["Coupon_Rate"].iloc[iCount])
    frequency = np.append(frequency,bondfile["Frequency"].iloc[iCount])
    recovrate = np.append(recovrate,bondfile["Recovery_Rate"].iloc[iCount])
    defprob = np.append(defprob,bondfile["Default_Probability"].iloc[iCount])
    sspread = np.append(sspread,bondfile["Sector_Spread"].iloc[iCount])

zspread = np.array([0.008, 0.008,0.005]) # This will be calibrated in a further development

In [9]:
# Example bond

#issuedate = np.array([dt.date(2021,12,3),dt.date(2021,12,3),dt.date(2019,12,3)])
#maturitydate = np.array([dt.date(2026,12,12),dt.date(2028,12,12),dt.date(2020,12,3)])
#notional = np.array([100,100,100])
#couponrate = np.array([0.03, 0.05,0.04])
#frequency = np.array([1, 1, 1])
#recovrate = np.array([0.4, 0.4,0.4]) 
#defprob =np.array([0.03, 0.03,0.04]) 
#sspread = np.array([0.008, 0.008,0.005])
#zspread = np.array([0.008, 0.008,0.005])

In [10]:
couponrate

array([0.03, 0.05, 0.04])

In [11]:
zcb = ZeroCouponBond(issuedate, maturitydate, frequency, notional, couponrate, recovrate, defprob, zspread, sspread)

In [12]:
zcb.createcashflows()

In [13]:
zcbPriced = ZeroCouponBondPriced(MD,zspread,compounding)

In [14]:
[coupondatesconsidered,coupondatefrac] = zcbPriced.refactordates(zcb.coupondates,MD)
[notionaldatesconsidered,notionaldatefrac] = zcbPriced.refactordates(zcb.notionaldates,MD)

In [15]:
# Numeric precision of the optimisation
Precision = float(paramfile.loc["Precision"][0])

# Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point
Tau = float(paramfile.loc["Tau"][0])# 1 basis point

Country = paramfile.loc["country"]

InitialDate = paramfile.loc["Modelling_Date"]

Curves = Curves(ufr, Precision, Tau, InitialDate, Country)

In [16]:
coupontargetrates = []
notionaltargetrates = []

for iAsset in range(0,issuedate.size-1):

    if not notionaldatesconsidered:
       print("This bond has matured") 
    else: 
        coupontargetrates.append(Curves.SWExtrapolate(np.transpose(coupondatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        notionaltargetrates.append(Curves.SWExtrapolate(np.transpose(notionaldatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        zcbPriced.coupondatefrac.append(np.transpose(coupondatefrac[iAsset]))
        zcbPriced.notionaldatefrac.append(np.transpose(notionaldatefrac[iAsset]))
        zcbPriced.couponcfs.append(zcb.couponcfs[iAsset][coupondatesconsidered[iAsset]])
        zcbPriced.notionalcfs.append(zcb.notionalcfs[iAsset])

Remove cashflows that are not considered

In [17]:
zcbPriced.PriceBond(coupontargetrates,notionaltargetrates,coupondatefrac,notionaldatefrac,zcbPriced.couponcfs,zcbPriced.notionalcfs)

### Price of two hypothetical bonds that mature after modelling date

In [18]:
zcbPriced.marketprice

[array([97.48139803]), array([107.1858606])]